In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import logging
import boto3
import botocore.exceptions
import datetime
import argparse
import warnings
import os, sys

df=pd.read_excel(r'C:\Users\soporte\Desktop\var.xlsx',names=[0,1,2,3]) ##my dataframe

host= "rs-bigdata-datalake-prd.c8evymt0o1da.us-east-1.redshift.amazonaws.com"
port=5439
dbname="analitico"
user="extcarlosmartinez"
password="Belcorp202121"

def createCursor(host, port, dbname, user, password):

    try:
        con = psycopg2.connect(host=host,
                               port=port,
                               dbname=dbname,
                               user=user,
                               password=str(password))

        con.autocommit = True

        return con.cursor()
    except:
        print('Unable to connect!\n{}'.format(psycopg2.InternalError))
        sys.exit(1)
        
def insert(df):
    cur = createCursor(host, port, dbname, user, password)
    column_names=list(df)
    for index, row in df.iterrows():
        campana=str(row[column_names[0]])
        pais=str(row[column_names[1]])
        zona=str(row[column_names[2]])
        if str(row[column_names[3]])=="D":
            delete_query="DELETE FROM sbx_analytics_optimization.zonas_fan_prd WHERE aniocampana="+"'"+campana+"'"+" and "+"codpais="+"'"+pais+"'"+ " and codzona="+ "'"+zona+"'"
            cur.execute(delete_query)
        else:
            count_query="SELECT COUNT(1) FROM sbx_analytics_optimization.zonas_fan_prd WHERE aniocampana="+"'"+campana+"'"+" and "+"codpais="+"'"+pais+"'"+ " and codzona="+ "'"+zona+"'"
            cur.execute(count_query)
            counter = pd.DataFrame(cur.fetchall())
            if counter[0].tolist()[0]==0:
                sql_query='INSERT INTO sbx_analytics_optimization.zonas_fan_prd VALUES (' + "'"+campana+ "'"+"," +"'"+ pais+"'"+","+"'"+zona +"'" +")"
                cur.execute(sql_query)
    cur.close()
try:
    insert(df)
except:
    print("error") ##error

